In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [2]:
test_df = pd.read_csv('clean_test.csv')
train_df = pd.read_csv('clean_train.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(8640, 14)
(4277, 13)


In [4]:
final_df = pd.concat([train_df,test_df],axis=0)
final_df.shape

(12917, 14)

In [5]:
def category_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        print(fields)
        df1 = pd.get_dummies(final_df[fields],drop_first=True)
        final_df.drop([fields],axis=1,inplace=True)
        if i==0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final,df1],axis=1)
        i = i + 1

    df_final = pd.concat([final_df,df_final],axis = 1)
    return df_final

In [6]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12917 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12917 non-null  object 
 1   HomePlanet    12917 non-null  object 
 2   CryoSleep     12917 non-null  bool   
 3   Cabin         12917 non-null  object 
 4   Destination   12917 non-null  object 
 5   Age           12917 non-null  float64
 6   VIP           12917 non-null  bool   
 7   RoomService   12917 non-null  float64
 8   FoodCourt     12917 non-null  float64
 9   ShoppingMall  12917 non-null  float64
 10  Spa           12917 non-null  float64
 11  VRDeck        12917 non-null  float64
 12  Name          12917 non-null  object 
 13  Transported   8640 non-null   object 
dtypes: bool(2), float64(6), object(6)
memory usage: 1.3+ MB


In [8]:
# cat_attributes = final_df.select_dtypes(include=['object'])

cat_attributes = final_df[['PassengerId', 'HomePlanet', 'Cabin', 'Destination', 'Name']]

In [9]:
final_df = category_onehot_multcols(cat_attributes)

PassengerId
HomePlanet
Cabin
Destination
Name


In [10]:
final_df.shape

(12917, 35310)

In [11]:
final_df = final_df.loc[:,~final_df.columns.duplicated()]
final_df.shape

(12917, 35309)

In [12]:
df_train = final_df.iloc[:8640,:]
df_test = final_df.iloc[8640:,:]

In [13]:
print(df_train.shape)
print(df_test.shape)

(8640, 35309)
(4277, 35309)


In [14]:
df_test.drop(['Transported'],axis=1,inplace=True)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
x_train = df_train.drop(['Transported'],axis=1)
y_train = df_train['Transported']

### First Model (Score : 0.0000)

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [17]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [18]:
model.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [20]:
import joblib

# Save model
joblib.dump(model, "xgb_model.pkl")

# Load saved model
# my_model_loaded = joblib.load("xgb_model.pkl")

['xgb_model.pkl']

In [21]:
model_pred = model.predict(df_test)

In [22]:
pred = pd.DataFrame(model_pred)
sub_df = pd.read_csv('../../Datasets/spaceship-titanic/sample_submission.csv')
datasets = pd.concat([sub_df['PassengerId'],pred],axis=1)

In [23]:
datasets.columns = ['PassengerId', 'Transported']

In [27]:
# Convert Tranported to boolean
datasets["Transported"] = datasets["Transported"].astype(bool)

In [29]:
datasets.to_csv('submission.csv',index=False)